In [17]:
# Importing essential libraries
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load environment variables for secure database connection
load_dotenv()


True

In [18]:
# Load database credentials from environment variables
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")

# Create the SQLAlchemy engine
connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)
print("Successfully connected to PostgreSQL using SQLAlchemy engine")


Successfully connected to PostgreSQL using SQLAlchemy engine


In [19]:
# Define the file path (adjust if necessary)
file_path = '../data/Chatbot_iOS_Apps.csv'  # Adjust the path if needed

# Confirm that the file can be loaded
df_ios = pd.read_csv(file_path)
print("Successfully loaded iOS API data")
df_ios.head()


Successfully loaded iOS API data


,app_name,app_store_url,description,ratingValue,reviewCount,category
0,ChatGPT,https://apps.apple.com/us/app/chatgpt/id644831...,Introducing ChatGPT for iOS: OpenAI’s latest a...,4.9,2390387,free
1,Chatbot AI Assistant - Genie,https://apps.apple.com/us/app/chatbot-ai-assis...,Genie is a revolutionary AI chatbot assistant ...,4.7,183014,free
2,AI Chatbot - Nova,https://apps.apple.com/us/app/ai-chatbot-nova/...,Nova is a revolutionary AI chatbot and persona...,4.7,91167,free
3,AI Chat Assistant Ð ChatNow,https://apps.apple.com/us/app/ai-chat-assistan...,ChatNow — Your All-in-One AI Assistant\n\nWelc...,4.9,78,free
4,AI Chatbot_,https://apps.apple.com/us/app/ai-chatbot/id644...,Welcome to AI Chatbot ChatPlus powered by GPT-...,4.8,865,free


## Business Question
**Which iOS chatbots have the highest average rating, and which are the most popular based on the number of user reviews?**

In [20]:
# Defining the descriptive analytics query with the correct table name
sql_query_descriptive = '''
SELECT 
    "app_name",
    AVG("ratingValue") AS avg_rating,
    SUM("reviewCount") AS total_reviews
FROM 
    "Chatbot iOS Apps"  
GROUP BY 
    "app_name"
ORDER BY 
    avg_rating DESC;
'''

# Running the descriptive query
try:
    df_descriptive = pd.read_sql(sql_query_descriptive, con=engine)
    print("Descriptive Query executed successfully")
    pd.set_option('display.max_rows', None)
    df_descriptive.head()
except Exception as e:
    print(f"Query failed: {e}")

Descriptive Query executed successfully


In [21]:
# Save the descriptive analysis results as a PostgreSQL table
try:
    df_descriptive.to_sql(
        'ios_chatbot_descriptive_analysis',  # Table name
        con=engine,
        if_exists='replace',  # Replace the table if it already exists
        index=False           # Do not save the index column
    )
    print("Descriptive Analysis results saved to PostgreSQL table: ios_chatbot_descriptive_analysis")
except Exception as e:
    print(f"Failed to save the table: {e}")


Descriptive Analysis results saved to PostgreSQL table: ios_chatbot_descriptive_analysis


In [22]:
# List all tables in your PostgreSQL database
table_check_query = '''
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public';
'''

tables = pd.read_sql(table_check_query, engine)
print("Available Tables in Database:")
print(tables)


Available Tables in Database:
                          table_name
0                   Chatbot iOS Apps
1   ios_chatbot_descriptive_analysis
2                chatbot_performance
3       chatbot_descriptive_analysis
4  13 Best Free and Paid AI Chatbots
5        chatbot_diagnostic_analysis


In [23]:
# Displaying the saved descriptive analysis table
sql_query_display = '''
SELECT * 
FROM ios_chatbot_descriptive_analysis;
'''

# Loading the table into a DataFrame
df_display = pd.read_sql(sql_query_display, con=engine)
pd.set_option('display.max_rows', None)
df_display.head()


,app_name,avg_rating,total_reviews
0,Ask GPT -4: Advance GPT AI Chat,5.0,4.0
1,ChatGPT,4.9,2390387.0
2,Perplexity - Ask Anything,4.9,169266.0
3,AI Chat Assistant Ð ChatNow,4.9,78.0
4,Grok by xAI,4.9,309764.0



## Insight
- The analysis shows that while several chatbots have high average ratings (4.8 - 5.0), the most popular chatbots are distinguished by their high number of user reviews.
- For example, ChatGPT has one of the highest ratings (4.9) and also the highest number of reviews (over 2.3 million), indicating it is both highly rated and widely used.
- Other chatbots, such as Perplexity - Ask Anything and AI Chat Assistant Ð ChatNow, also maintain high ratings (4.9) with substantial user bases.
- Conversely, some chatbots like Ask GPT -4: Advance GPT AI Chat have a perfect rating (5.0) but very few reviews, which suggests these ratings may not be reliable due to a lack of user feedback.
- The analysis reveals that a high average rating alone is not enough — popularity (measured by review count) is a strong indicator of trust and widespread use.


## Recommendation
- For companies looking to integrate a chatbot into their platforms:
  - Prioritize chatbots with a combination of high ratings and high user reviews. These are the most tested and reliable.
  - If user satisfaction is the priority, focus on chatbots with consistently high ratings, even if they have fewer reviews.
  - Consider testing chatbots like ChatGPT, Perplexity, and AI Chat Assistant Ð ChatNow, which balances high ratings with large user bases.
  - For emerging chatbots with high ratings but low reviews, monitor their growth to see if they maintain positive feedback as more users adopt them.

## Prediction
- Chatbots with high average ratings and a high number of user reviews will continue to dominate the market, as they demonstrate both user satisfaction and widespread adoption.
- Chatbots with very high ratings but low review counts may see their ratings drop as more users provide feedback, revealing more accurate performance metrics.
- If a new chatbot quickly gains a high number of reviews while maintaining a high rating, it may become a strong competitor to established apps like ChatGPT.

## Business Question
**Which iOS chatbots provide the best balance between high user ratings and popularity (measured by review count)?**

### Understanding the Three Rankings:
- Rating Rank: Ranks chatbots by their average user rating (higher rating = higher rank).  
- Popularity Rank: Ranks chatbots by their total number of reviews (more reviews = higher rank).  
- Combined Score: A combined rank of both Rating and Popularity, calculated as: combined_score = rating_rank + popularity_rank
- A lower combined score means a stronger balance of high rating and high popularity.

In [25]:
# Defining the diagnostic analytics query: Performance vs. Popularity
sql_query_diagnostic = '''
WITH PerformancePopularity AS (
    SELECT 
        "app_name",
        "ratingValue",
        "reviewCount",
        RANK() OVER (ORDER BY "ratingValue" DESC) AS rating_rank,
        RANK() OVER (ORDER BY "reviewCount" DESC) AS popularity_rank,
        (RANK() OVER (ORDER BY "ratingValue" DESC) + 
         RANK() OVER (ORDER BY "reviewCount" DESC)) AS combined_score
    FROM 
        "Chatbot iOS Apps"
)
SELECT 
    "app_name",
    "ratingValue",
    "reviewCount",
    rating_rank,
    popularity_rank,
    combined_score
FROM 
    PerformancePopularity
ORDER BY 
    combined_score ASC;  -- Lower combined score means higher combined rank
'''

# Running the diagnostic query
try:
    df_diagnostic = pd.read_sql(sql_query_diagnostic, con=engine)
    print("Diagnostic Query executed successfully")
    pd.set_option('display.max_rows', None)
    df_diagnostic.head()
except Exception as e:
    print(f"Query failed: {e}")


Diagnostic Query executed successfully


In [26]:
# Save the diagnostic analysis results as a PostgreSQL table
try:
    df_diagnostic.to_sql(
        'ios_chatbot_diagnostic_analysis',  # Table name
        con=engine,
        if_exists='replace',
        index=False
    )
    print("Diagnostic Analysis results saved to PostgreSQL table: ios_chatbot_diagnostic_analysis")
except Exception as e:
    print(f"Failed to save the table: {e}")


Diagnostic Analysis results saved to PostgreSQL table: ios_chatbot_diagnostic_analysis


In [27]:
# Displaying the saved diagnostic analysis table
sql_query_display = '''
SELECT * 
FROM ios_chatbot_diagnostic_analysis;
'''

# Loading the table into a DataFrame
df_diagnostic_display = pd.read_sql(sql_query_display, con=engine)
pd.set_option('display.max_rows', None)
df_diagnostic_display.head()

,app_name,ratingValue,reviewCount,rating_rank,popularity_rank,combined_score
0,ChatGPT,4.9,2390387,2,1,3
1,Grok by xAI,4.9,309764,2,3,5
2,Chat & Ask AI by Codeway,4.8,305285,6,4,10
3,Google Gemini,4.8,229130,6,5,11
4,Perplexity - Ask Anything,4.9,169266,2,12,14


## Insight
- The analysis reveals that while some chatbots maintain high ratings, they may not be widely used, while others are extremely popular but have slightly lower ratings.
- ChatGPT consistently ranks among the top performers, with both a high rating and the highest review count (over 2.3 million).
- Other chatbots like Perplexity - Ask Anything** and **AI Chat Assistant Ð ChatNow maintain high ratings with significant user bases, indicating strong user satisfaction and popularity.
- Chatbots with high ratings but low review counts (like "Ask GPT -4: Advance GPT AI Chat") are likely newer or less well-known, and their ratings may change as they gain more users.


## Recommendation
- For companies looking to integrate a chatbot:
- Prioritize chatbots with a balanced performance, like ChatGPT, Perplexity, and AI Chat Assistant Ð ChatNow, which offer high user satisfaction and have proven popularity.
- Monitor chatbots with high ratings but low review counts as "emerging options" — if they maintain their ratings with more reviews, they could become strong contenders.
- Avoid chatbots with low ratings or high review counts with poor ratings, as they indicate consistent user dissatisfaction.


## Prediction
- Chatbots with a high combined ranking (high rating + high review count) will continue to dominate the market as they balance quality and user trust.
- Chatbots with high ratings but low review counts may see their ratings drop as they become more popular and receive more feedback.
- New chatbots with strong initial ratings but low review counts may gain popularity quickly if they maintain user satisfaction.